# 1.0 模块加载

In [18]:
import import_ipynb
import oss2
from setting import SETTINGS
import pickle
import pandas as pd
import os
import io
from dateutil import parser
import datetime
import pandas as pd
from fastparquet import write
from pathlib import  Path
from oss_handler import OssClient
import sys, os
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from pylab import mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime
from datetime import datetime, timedelta
from oss2.exceptions import NoSuchKey
import warnings

warnings.filterwarnings('ignore')

AccessKeyId = SETTINGS["oss.accesskey"]
AccessKeySecret = SETTINGS["oss.secret"]
BucketName = SETTINGS["oss.bucketname"]
Endpoint = SETTINGS["oss.endpoint"]


class newBytes(io.BytesIO):
    def close(self):
        pass


class OssClient(object):
    __instance = None
    __first_init = False

    # 单例模式
    def __new__(cls, *args, **kwargs):
        if not cls.__instance:
            cls.__instance = super().__new__(cls)
        return cls.__instance

    def __init__(self):
        cls = self.__class__
        if not cls.__first_init:
            self.auth = oss2.Auth(AccessKeyId, AccessKeySecret)
            self.bucket = oss2.Bucket(self.auth, Endpoint, BucketName)
            cls.__first_init = True


    def upload_file_from_fileobj(self, object_name, local_file_path):
        """
            upload_file_from_fileobj方法：上传文件对象到oss存储空间, 该方法可用于我们从上游服务接收了图片参数，然后以二进制形式读文件，上传到oss存储空间指定位置（abc/efg/00），
        当然也可以将本地文件上传到oss我们的bucket. 其中fileobj不止可以是文件对象，也可以是本地文件路径。 put_object方法底层仍是RESTful API的调用，可以指定headers，规定Content-Type等内容
        """
        # 判断bucket中文件是否存在，也可以不判断，会上传更新
        #exist = self.bucket.object_exists(object_name) #<yourObjectName>
        #if exist:
        #    return True
        with open(local_file_path, 'rb') as fileobj:
            result = self.bucket.put_object(object_name, fileobj) #<yourObjectName>
        if result.status == 200:
            return True
        else:
            return False

    def upload_pickle_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        pickle_buffer = io.BytesIO()
        pickle.dump(df, pickle_buffer)
        target_file_key = os.path.join(target_path, '{}.pkl'.format(d)).replace("\\","/")
        result = self.bucket.put_object(target_file_key, pickle_buffer.getvalue())
        if result.status == 200:
            return True
        else:
            return False


    def list_files(self,prefix = None):
        res = []
        for object_info in oss2.ObjectIterator(self.bucket,prefix):
            print(object_info.key)
            res.append(object_info.key)
        return res

    def upload_parquet_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(target_path, '{}.parquet'.format(d)).replace("\\","/")
        mem_buffer = newBytes()
        df.to_parquet('noname', engine='fastparquet', open_with=lambda x, y: mem_buffer)
        result = self.bucket.put_object(target_file_key, mem_buffer.getvalue())
        #f = Path(os.getcwd())/'tmp.parquet'
        #write(f, df)
        #with open(f, 'rb') as fileobj:
        #    result = self.bucket.put_object(target_file_key, fileobj)
        #os.remove(f)
        if result.status == 200:
            return True
        else:
            return False

    def save_data_to_pickle(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, '{}.pkl'.format(d))
        with open(target_file_key, 'wb') as f:
            pickle.dump(df, f)

    def save_data_to_parquet(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, f'{d}.parquet')
        df.to_parquet(target_file_key)


    def read_oss_pickle_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pickle.loads(result)

    def read_oss_parquet_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        i = io.BytesIO(result)
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pd.read_parquet(i)


    def download_file_to_loaclfilepath(self, object_name, local_file_path):
        """
            download_file_to_loaclfilepath：下载文件到本地路径。get_object和get_object_to_file的区别是前者是获取文件流实例，可用于代码处理和远程调用参赛。后者是存储到本地路径，返回的是一个http状态的json结果
        """
        result = self.bucket.get_object_to_file(object_name, local_file_path) # ('<yourObjectName>', '<yourLocalFile>')
        if result.status == 200:
            return True
        else:
            return False

    def generate_temporary_download_url(self,object_name):
        """
            generate_temporary_download_url: 生成加签的临时URL以供授信用户下载。一般在实际业务中，我们是提供给调用方一个临时下载链接，来让其获取文件数据，而不是直接使用以上暴露AccessKeyId和AccessKeySecret的方法。
            因此一般我们会存储某条数据oss的路径（<yourObjectName>）与调用方某个唯一标识的对应关系（如手机号身份证号），在调用方请求时，通过该标识获取其数据的oss文件路径（<yourObjectName>），
            然后制定过期时间，为其生成临时下载链接
            http://bucketname.oss-ap-south-1.aliyuncs.com/abc/efg/0?OSSAccessKeyId=LTA************oN9&Expires=1604638842&Signature=tPgvWz*************Uk%3D
        """
        res_temporary_url = self.bucket.sign_url('GET', object_name, 60, slash_safe=True)
        return res_temporary_url

oss_client = OssClient()

# $=========手动输入=========$

In [1032]:
# 选中的股票
selected_stocks = [
    '300953',
    '300611',
    '301368',
    '001319',
    '688306'
]

# 实际股票数量
stock_positions = [
    700, 
    600, 
    1000, 
    400, 
    500
]

# 总资金
total_capital = 1000000

# $=========手动输入=========$

# 2.0 数据加载

In [1033]:
# 指数数据
# idx000001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000001.XSHG_hist_index.pkl') #上证指数
# idx000016 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000016.XSHG_hist_index.pkl') #上证 50
idx000300 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000300.XSHG_hist_index.pkl') #沪深 300
idx000852 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000852.XSHG_hist_index.pkl') #中证 1000
# idx000905 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000905.XSHG_hist_index.pkl') #中证 500
沪深京全A = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000985.XSHG_hist_index.pkl') #中证全指
# idx399001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399001.XSHE_hist_index.pkl') #深证成指
# idx399006 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399006.XSHG_hist_index.pkl') #创业板指
# idx399303 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399303.XSHG_hist_index.pkl') #国证 2000

# 个股数据
adj_close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_close.pk')
adj_open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_open.pk')
adj_high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_high.pk')
adj_low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_low.pk')
# Close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/close.pk') 
# Open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/open.pk') 

# limit_down = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_down.pk') #跌停
# limit_up = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_up.pk') #涨停

# halt_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/halt_status.pk') #停牌 
# st_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/st_status.pk') #st
# st_status = st_status.replace(True,np.nan).replace(False,True)
# halt_status = halt_status.replace(True,np.nan).replace(False,True)

adj_return =  adj_close/adj_close.shift(1) - 1
adj_return = adj_return.replace(np.inf,np.nan).replace(-np.inf,np.nan)
adj_return = adj_return[(adj_return.index >= '2017-01-01')]

# 3.0 大盘判断

## 3.1 主升浪 （80% - 100%仓位）

### 3.1.1 大盘全指判断

In [1034]:
#条件一 ： 大于20日平均成交额
沪深京全A['成交额2万亿20日mean'] = 沪深京全A['total_turnover'].rolling(20).mean()
沪深京全A['成交额大于20日mean'] = np.where(沪深京全A['total_turnover'] >= 沪深京全A['成交额2万亿20日mean'], 1, 0)

#条件二 ： 大于2万亿
沪深京全A['成交额2万亿'] = np.where(沪深京全A['total_turnover'] >= 2e+12, 1, 0) 

#判断
沪深京全A['全A放量'] = np.where(沪深京全A['成交额大于20日mean'] + 沪深京全A['成交额2万亿'] >= 1, 1, 0)

### 3.1.2 沪深300/中证1000指数判断

In [1035]:
def ma(index):
    index['MA5'] = index['close'].rolling(5).mean()
    index['MA10'] = index['close'].rolling(10).mean()
    index['MA20'] = index['close'].rolling(20).mean()
    return index
    
idx000300 = ma(idx000300)
idx000852 = ma(idx000852)

#### 3.1.2.1 沪深300

In [1036]:
#条件一 ： 多头排列
idx000300['多头排列'] = np.where(
    (idx000300['MA5'] >= idx000300['MA10']) & 
    (idx000300['MA10'] >= idx000300['MA20']),
    1,
    0
)

#条件二 ： 站在5日线以上且5日创新高
idx000300['5日线以上'] = np.where(idx000300['close'] >= idx000300['MA5'], 1, 0)

idx000300['Rolling_Max_5D'] = idx000300['high'].rolling(window=5).max()
idx000300['Is_New_High'] = (idx000300['high'] == idx000300['Rolling_Max_5D']).astype(int)
idx000300['连续创新高次数'] = idx000300['Is_New_High'].rolling(window=5).sum()
idx000300['连续5日创新高'] = np.where(idx000300['连续创新高次数'] == 5, 1, 0)

#判断
idx000300['站稳5日线且创新高阶段'] = np.where(idx000300['5日线以上'] + idx000300['连续5日创新高'] >= 1, 1, 0)

#### 3.1.2.2 中证1000

In [1037]:
#中证1000

#条件一 ： 多头排列
idx000852['多头排列'] = np.where(
    (idx000852['MA5'] >= idx000852['MA10']) & 
    (idx000852['MA10'] >= idx000852['MA20']),
    1,
    0
)

#条件二 ： 站在5日线以上且5日创新高
idx000852['5日线以上'] = np.where(idx000852['close'] >= idx000852['MA5'], 1, 0)

idx000852['Rolling_Max_5D'] = idx000852['high'].rolling(window=5).max()
idx000852['Is_New_High'] = (idx000852['high'] == idx000852['Rolling_Max_5D']).astype(int)
idx000852['连续创新高次数'] = idx000852['Is_New_High'].rolling(window=5).sum()
idx000852['连续5日创新高'] = np.where(idx000852['连续创新高次数'] == 5, 1, 0)

#判断
idx000852['站稳5日线且创新高阶段'] = np.where(idx000852['5日线以上'] + idx000852['连续5日创新高'] >= 1, 1, 0)

### 3.1.2 综合判断，达到1分重仓8成以上

In [1038]:
沪深京全A['沪深300_重仓'] = np.where(沪深京全A['全A放量'] + idx000300['多头排列'] + idx000300['站稳5日线且创新高阶段'] >= 3, 1, 0)
沪深京全A['中证1000_重仓'] = np.where(沪深京全A['全A放量'] + idx000852['多头排列'] + idx000852['站稳5日线且创新高阶段'] >= 3, 1, 0)
判断_重仓 = 沪深京全A[['date','沪深300_重仓','中证1000_重仓']]

## 3.2 震荡市 （50% - 60%仓位）

### 3.2.1 大盘全指判断

In [1039]:
#条件一 ： 大于1.5万亿
沪深京全A['成交额1.5万亿'] = np.where(沪深京全A['total_turnover'] >= 1.5e+12, 1, 0) 

### 3.2.2 沪深300/中证1000指数震荡判断

#### 3.2.2.1 沪深300

In [1040]:
#条件一 ： 多头排列
idx000852['多头排列']

#条件二 ： 站在20日线以上
idx000300['20日线以上'] = np.where(idx000300['close'] >= idx000300['MA20'], 1, 0)

#判断
idx000300['站上20日线且多头排列'] = np.where(idx000300['20日线以上'] + idx000300['多头排列'] >= 2, 1, 0)

#### 3.2.2.2 中证1000

In [1041]:
#条件一 ： 多头排列
idx000852['多头排列']

#条件二 ： 站在20日线以上
idx000852['20日线以上'] = np.where(idx000852['close'] >= idx000852['MA20'], 1, 0)

#判断
idx000852['站上20日线且多头排列'] = np.where(idx000852['20日线以上'] + idx000852['多头排列'] >= 2, 1, 0)

### 3.2.3 综合判断，达到1分半仓

In [1042]:
沪深京全A['沪深300_半仓'] = np.where(沪深京全A['成交额1.5万亿'] + idx000300['站上20日线且多头排列'] >= 2, 1, 0)
沪深京全A['中证1000_半仓'] = np.where(沪深京全A['成交额1.5万亿'] + idx000852['站上20日线且多头排列'] >= 2, 1, 0)
判断_半仓 = 沪深京全A[['date','沪深300_半仓','中证1000_半仓']]

## 3.3 下跌 （0% - 20%仓位）

### 3.3.1 沪深300/中证1000下跌判断

#### 3.3.1.1 沪深300

In [1043]:
# 条件一 ：指数在5日线以下
idx000300['5日线以下'] = np.where(idx000300['close'] <= idx000300['MA5'], 1, 0)

# 条件二 ：5日10日死叉
idx000300['5日10日死叉'] = np.where(idx000300['MA5'] <= idx000300['MA10'], 1, 0)

# 条件三 ：跌破20日线
idx000300['20日线以上'] = np.where(idx000300['close'] >= idx000300['MA20'], 1, 0)

#判断
idx000300['轻仓判断'] = np.where(idx000300['5日线以下'] + idx000300['5日10日死叉'] + idx000300['20日线以上']>= 3, 1, 0)

### 3.3.1.2 中证1000

In [1044]:
# 条件一 ：指数在5日线以下
idx000852['5日线以下'] = np.where(idx000852['close'] <= idx000852['MA5'], 1, 0)

# 条件二 ：5日10日死叉
idx000852['5日10日死叉'] = np.where(idx000852['MA5'] <= idx000852['MA10'], 1, 0)

# 条件三 ：站稳20日线
idx000852['20日线以上'] = np.where(idx000852['close'] >= idx000852['MA20'], 1, 0)

#判断
idx000852['轻仓判断'] = np.where(idx000852['5日线以下'] + idx000852['5日10日死叉'] + idx000852['20日线以上']>= 3, 1, 0)

## 3.4 沪深300/中证1000空仓判断

### 3.4.1 沪深300

In [1045]:
# 条件一 ：跌破20日线
idx000300['跌破20日线'] = np.where(idx000300['close'] < idx000300['MA20'], 1, 0)

### 3.4.2 中证1000

In [1046]:
# 条件一 ：跌破20日线
idx000852['跌破20日线'] = np.where(idx000852['close'] < idx000852['MA20'], 1, 0)

### 3.4.3 综合判断，达到清仓空仓

In [1047]:
沪深京全A['沪深300_轻仓'] = idx000300['轻仓判断']
沪深京全A['中证1000_轻仓'] = idx000852['轻仓判断']
沪深京全A['沪深300_空仓'] = idx000300['跌破20日线']
沪深京全A['中证1000_空仓'] = idx000852['跌破20日线']
判断_轻仓 = 沪深京全A[['date','沪深300_轻仓','中证1000_轻仓','沪深300_空仓','中证1000_空仓']]
判断_轻仓.tail(1)

,date,沪深300_轻仓,中证1000_轻仓,沪深300_空仓,中证1000_空仓
3927,2025-03-07,0,0,0,0


## 3.5 最终判断

In [1089]:
大盘综合判断 = 沪深京全A[['date','成交额2万亿20日mean', '成交额大于20日mean',
       '成交额2万亿', '全A放量', '沪深300_重仓','沪深300_半仓','沪深300_轻仓','沪深300_空仓',
              '中证1000_重仓','中证1000_半仓','中证1000_轻仓','中证1000_空仓']]

# 4.0 个股仓位管理

### 4.1 ATR个股仓位

In [1050]:
## 条件一 ： 20日平均波动率 TR=max(H−L,∣H−Cprev ∣,∣L−Cprev∣)
a = (np.abs(adj_high -adj_low)).iloc[1:,:]
b = (np.abs(adj_high -adj_close.shift(1))).iloc[1:,:]
c = (np.abs(adj_low -adj_close.shift(1))).iloc[1:,:]

x = a.stack().reset_index()
y = b.stack().reset_index()
z = c.stack().reset_index()

merged = pd.merge(x, y, on=['date', 'ticker'], how='outer')  # 或者 'inner'，根据需要选择
merged = pd.merge(merged, z, on=['date', 'ticker'], how='outer')
merged.set_index(['date', 'ticker'], inplace=True)

merged.columns = ['a', 'b', 'c']
merged['TR'] = merged[['a','b','c']].max(axis=1)
merged = merged.drop(columns=['a','b','c'])
merged['ATR_init'] = merged.sort_values(['ticker', 'date']).groupby('ticker')['TR'].transform(lambda x: x.rolling(window=20).mean())
merged['ATR_init_form'] = np.where(merged['ATR_init'].notna(), 1, 0)
merged['TR_new'] = merged['TR'] * merged['ATR_init_form'].replace(0, np.nan)

N = 20
lambda_20 = 2/(N+1)

merged['ATR_pre1'] = merged.groupby('ticker')['ATR_init'].transform(lambda x: x.shift(1)*(1- lambda_20))
merged['TR_lambda'] = merged.groupby('ticker')['TR_new'].transform(lambda x: x*lambda_20)
merged = merged.drop(columns=['TR','ATR_init','ATR_init_form','TR_new'])
merged['ATRt_1d'] = merged['ATR_pre1'] + merged['TR_lambda']
merged_pure = merged.ffill().dropna(axis=0)
merged_pure['ATR'] = np.nan
merged_pure['mask_first'] = merged_pure.groupby('ticker').cumcount() == 0
merged_pure['ATR'] = (merged_pure['ATRt_1d'] * merged_pure['mask_first']).replace(0,np.nan)
merged_pure = merged_pure.drop(columns=['ATR_pre1','ATRt_1d','mask_first'])

TR_lambda = (merged_pure['TR_lambda']).unstack()
ATR_init = (merged_pure['ATR']).unstack()
ATR = pd.read_pickle('D:/redata/factor/ATR@factor.pkl')

empty = (TR_lambda-TR_lambda)
empty[:] = 1
ATR = ATR * empty

In [1051]:
# 单天打标 #
ATR_updated = ATR.copy()
mask = ~TR_lambda.isna() & ATR_updated.notna().shift(1, axis=0).fillna(False)
ATR_updated.values[1:, :] = np.where(
    mask.values[1:, :],
    lambda_20 * TR_lambda.values[1:, :] + (1 - lambda_20) * ATR_updated.values[:-1, :],
    ATR_updated.values[1:, :])
ATR_updated.to_pickle('D:/redata/factor/ATR@factor.pkl')

In [1052]:
# 多天计算 #
# 初始化 ATR_updated 为 ATR 的副本
# ATR_updated_cp = ATR.iloc[100:,:]
# TR_lambda_cp = TR_lambda.iloc[100:,:]
# max_length = len(TR_lambda_cp)

# # 设置最大迭代次数以防止潜在的无限循环
# max_iterations = max_length -1
# iteration = 0

# # 循环直到没有 NaN 或达到最大迭代次数
# while iteration < max_iterations:
#     # 创建掩码
#     mask = ~TR_lambda_cp.isna() & ATR_updated_cp.notna().shift(1, axis=0).fillna(False)
    
#     # 找到需要更新的行
#     rows_to_update = mask.values[1:, :]  # 从第二行开始
    
#     if not np.any(rows_to_update):
#         # 如果没有行需要更新，退出循环
#         break
    
#     # 计算新的 ATR 值
#     new_values = lambda_20 * TR_lambda_cp.values[1:, :] + (1 - lambda_20) * ATR_updated_cp.values[:-1, :]
    
#     # 更新 ATR_updated
#     ATR_updated_cp.values[1:, :][rows_to_update] = new_values[rows_to_update]
    
#     # 检查是否还有 NaN
#     if not np.isnan(ATR_updated).any().any():
#         break
    
#     iteration += 1

# ATR_updated.to_pickle('D:/redata/factor/ATR@factor.pkl')

### 4.1.1 个股ATR Decision

In [1053]:
# selected_stocks = ['000001','000002','000006','000004','000007']
# selected_stocks = sorted(selected_stocks, reverse=False)

In [1067]:
selected_stocks = sorted(selected_stocks, reverse=False)
selected_stocks_atr = ATR[selected_stocks].iloc[-1:,:]
selected_stocks_atr = pd.DataFrame(selected_stocks_atr.stack())

selected_stocks_atr.columns = ['ATR']

conditions = [
    selected_stocks_atr['ATR'] <= 0.10,   # 条件1: ATR ≤ 0.10
    selected_stocks_atr['ATR'] >= 0.15    # 条件2: ATR ≥ 0.15
]

choices = [
    0.20,   # 条件1满足时赋值0.20
    0.05    # 条件2满足时赋值0.05
]

# 默认值
default = 0.10

# 使用 numpy.select 进行条件赋值
selected_stocks_atr['买入最大仓位'] = (np.select(conditions, choices, default)) * 100
selected_stocks_atr['买入最大仓位'] = selected_stocks_atr['买入最大仓位'].apply(lambda x: f"{x:.2f}%")


selected_stocks_atr

ATR  买入最大仓位
date       ticker                  
2025-03-07 001319  0.168177   5.00%
           300611  0.179731   5.00%
           300953  1.256611   5.00%
           301368  0.744476   5.00%
           688306  0.100949  10.00%

### 4.1.2 个股收盘价与5日线偏离度

In [1094]:
def reform(selected_stocks, data):
    df = data[selected_stocks].iloc[-1:,:]
    df = pd.DataFrame(df.stack())

selected_stock_adj_close = adj_close[selected_stocks].iloc[-1:,:]
selected_stock_adj_close = pd.DataFrame(selected_stock_adj_close.stack())
selected_stock_adj_close.columns = ['adj_close']

stock_ma5 = adj_close.rolling(5).mean()
selected_stocks_ma5 = stock_ma5[selected_stocks].iloc[-1:,:]
selected_stocks_ma5 = pd.DataFrame(selected_stocks_ma5.stack())
selected_stocks_ma5.columns = ['ma5']
selected_stock_adj_close['ma5'] = selected_stocks_ma5['ma5']
selected_stock_adj_close['bias'] = (((selected_stock_adj_close['adj_close'] - 
                                      selected_stock_adj_close['ma5'])/ 
                                     selected_stock_adj_close['ma5'])).round(4)

selected_stock_bias_5 = pd.DataFrame(selected_stock_adj_close['bias'])

In [1095]:
# 条件一 偏离度小于0.05

# conditions_bias = [
#     (selected_stock_bias_5['bias'] <= 0.05) & (selected_stock_bias_5['bias'] >= -0.05)]

# choices_bias = [1]


# selected_stock_bias_5['sign_Bias5'] = (np.select(conditions_bias, choices_bias, default=0))

# 条件一 买入点偏离度小于0.05

selected_stocks_ma5['买入点max'] = (selected_stocks_ma5['ma5'] * 1.05).round(2)
selected_stocks_ma5['买入点low'] = (selected_stocks_ma5['ma5'] * 0.95).round(2)

In [1096]:
# 条件二 当日收盘涨跌幅小于5%
intra_rtn = (adj_close - adj_open)/adj_open 
selected_stock_intra_rtn = intra_rtn[selected_stocks].iloc[-1:,:]
selected_stock_intra_rtn = pd.DataFrame(selected_stock_intra_rtn.stack())
selected_stock_intra_rtn.columns = ['intra_return']

conditions_intra_rtn = [
    (selected_stock_intra_rtn['intra_return'] <= 0.05)]

choices_intra_rtn = [1]


selected_stock_intra_rtn['sign_DatRtn'] = (np.select(conditions_intra_rtn, choices_intra_rtn, default= 0))

In [1097]:
# 条件三 当日止损点
max_loss = total_capital * 0.01
stock_positions = [600, 600, 100, 400, 500]
selected_stock_intra_rtn['实际仓位'] = stock_positions # 持仓股票数 手动输入

# 计算每只股票的最大止损价
selected_stock_intra_rtn['止损max'] = (max_loss / selected_stock_intra_rtn['实际仓位']).round(2)

In [1102]:
# 最终个股信号
selected_stock_final_decision = pd.concat([selected_stocks_ma5, selected_stock_intra_rtn, selected_stocks_atr], axis=1)
# selected_stock_final_decision['最终交易信号'] = selected_stock_final_decision['sign_Bias5'] * selected_stock_final_decision['sign_DatRtn']
selected_stock_final_decision['最终交易信号'] = selected_stock_final_decision['sign_DatRtn']
selected_stock_final_decision = selected_stock_final_decision.drop(columns=['sign_DatRtn'])
selected_stock_final_decision = selected_stock_final_decision.replace(1,'买入')
selected_stock_final_decision = selected_stock_final_decision.replace(0,'等待回落')
selected_stock_final_decision = selected_stock_final_decision[['实际仓位','买入点max','买入点low','买入最大仓位','止损max','最终交易信号']]

In [1103]:
selected_stock_final_decision

实际仓位  买入点max  买入点low  买入最大仓位   止损max 最终交易信号
date       ticker                                             
2025-03-07 001319   600   30.68   27.76   5.00%   16.67   等待回落
           300611   600   21.22   19.20   5.00%   16.67   等待回落
           300953   100  155.90  141.06   5.00%  100.00   等待回落
           301368   400   90.50   81.88   5.00%   25.00   等待回落
           688306   500   12.76   11.55  10.00%   20.00   等待回落

In [1093]:
大盘综合判断.tail(2)

,date,成交额2万亿20日mean,成交额大于20日mean,成交额2万亿,全A放量,沪深300_重仓,沪深300_半仓,沪深300_轻仓,沪深300_空仓,中证1000_重仓,中证1000_半仓,中证1000_轻仓,中证1000_空仓
3926,2025-03-06,1.804308e+12,1,0,1,0,0,0,0,0,0,0,0
3927,2025-03-07,1.797471e+12,1,0,1,0,0,0,0,0,0,0,0
